In [1]:
import os
import json
import time
from collections import OrderedDict
import unittest
from datetime import datetime
from collections import namedtuple
import portpicker
from pytezos.contract.interface import ContractInterface
from pytezos.operation import MAX_OPERATIONS_TTL
import pytezos

PROJECT_ROOT = os.getcwd()
CHECKER_DIR = os.getenv(
    "CHECKER_DIR", default=os.path.join(PROJECT_ROOT, "checker-e2eTestsHack")
)
os.environ["PATH"] = f"{os.environ['PATH']}:{PROJECT_ROOT}/bin"
WRITE_GAS_COSTS = os.getenv("WRITE_GAS_COSTS")

from checker_client.checker import *


self = namedtuple("SELF", "client")
self.client = pytezos.pytezos.using("http://localhost:18742", key="edsk3RFfvaFaxbHx8BMtEW1rKQcPtDML3LXjNqMNLCzC3wLC1bWbAt")

In [2]:
print("Deploying the mock oracle.")
oracle = deploy_contract(
    self.client,
    source_file=os.path.join(PROJECT_ROOT, "util/mock_oracle.tz"),
    initial_storage=(self.client.key.public_key_hash(), 1000000),
    ttl=MAX_OPERATIONS_TTL,
)

print("Deploying ctez contract.")
ctez = deploy_ctez(
    self.client,
    ctez_dir=os.path.join(PROJECT_ROOT, "vendor/ctez"),
    ttl=MAX_OPERATIONS_TTL,
)

print("Deploying Checker.")
checker = deploy_checker(
    self.client,
    checker_dir=CHECKER_DIR,
    oracle=oracle.context.address,
    ctez=ctez["fa12_ctez"].context.address,
    ttl=MAX_OPERATIONS_TTL,
)

print("Deployment finished.")



Deploying the mock oracle.
Deploying ctez contract.
Deploying ctez contract...
Done.
Deploying ctez FA1.2 contract...
Done.
Deploying ctez CFMM contract...
Done.
Deploying liquidity contract...
Done.
Setting liquidity address in CFMM contract...
Done.
Setting CFMM amd ctez FA1.2 addresses in ctez contract...
Done.
Deploying Checker.
Deploying the wrapper.
Checker address: KT1GosH9zvtJwPFwGpwMbznmxxPwUZtu2B5B
Deploying the TZIP-16 metadata.
Deploying TZIP-16 metadata: chunk 1 of 8
Deploying TZIP-16 metadata: chunk 2 of 8
Deploying TZIP-16 metadata: chunk 3 of 8
Deploying TZIP-16 metadata: chunk 4 of 8
Deploying TZIP-16 metadata: chunk 5 of 8
Deploying TZIP-16 metadata: chunk 6 of 8
Deploying TZIP-16 metadata: chunk 7 of 8
Deploying TZIP-16 metadata: chunk 8 of 8
Deploying: deposit_tez
  deployed: chunk 0.
Deploying: activate_burrow
  deployed: chunk 0.
Deploying: sell_kit
  deployed: chunk 0.
Deploying: set_burrow_delegate
  deployed: chunk 0.
Deploying: mint_kit
  deployed: chunk 0.
De

In [3]:
def call_endpoint(contract, name, param, amount=0):
    print("Calling", contract.key.public_key_hash(), "/", name, "with", param)
    return inject(
        self.client,
        getattr(contract, name)(param)
        .with_amount(amount)
        .as_transaction()
        .autofill(ttl=MAX_OPERATIONS_TTL)
        .sign(),
    )

def call_bulk(bulks, *, batch_size):
    batches = [
        bulks[i : i + batch_size] for i in range(0, len(bulks), batch_size)
    ]
    times = []
    for batch_no, batch in enumerate(batches, 1):
        print(
            "Sending",
            len(batches),
            "operations as bulk:",
            "Batch",
            batch_no,
            "of",
            len(batches),
        )
        start = time.time()
        inject(
            self.client,
            self.client.bulk(*batch).autofill(ttl=MAX_OPERATIONS_TTL).sign(),
        )
        times.append(time.time() - start)
    print(f"Times: {times}")

In [32]:
call_bulk(
    [
        checker.create_burrow((0, None)).with_amount(200_000_000),
        checker.mint_kit((0, 80_000_000)),
    ],
    batch_size=10,
)

call_endpoint(
    ctez["ctez"], "create", (1, None, {"any": None}), amount=2_000_000
)
call_endpoint(ctez["ctez"], "mint_or_burn", (1, 100_000))
call_endpoint(ctez["fa12_ctez"], "approve", (checker.context.address, 100_000))

call_endpoint(
    checker,
    "add_liquidity",
    (100_000, 100_000, 5, int(datetime.now().timestamp()) + 20),
)

burrows = list(range(1, 1001))

call_bulk(
    [
        checker.create_burrow((burrow_id, None)).with_amount(100_000_000)
        for burrow_id in burrows
    ],
    batch_size=100,
)

# Mint as much as possible from the burrows. All should be identical, so we just query the
# first burrow and mint that much kit from all of them.
max_mintable_kit = checker.metadata.burrowMaxMintableKit(
    (self.client.key.public_key_hash(), 1)
).storage_view()

call_bulk(
    [checker.mint_kit(burrow_no, max_mintable_kit) for burrow_no in burrows],
    batch_size=100,
)

# Change the index (kits are 100x valuable)
#
# Keep in mind that we're using a patched checker on tests where the protected index
# is much faster to update.
call_endpoint(oracle, "update", 100_000_000)

# Oracle updates lag one touch on checker
call_endpoint(checker, "touch", None)
call_endpoint(checker, "touch", None)
time.sleep(20)
call_endpoint(checker, "touch", None)

# Now burrows should be overburrowed, so we liquidate them all.
#
# This should use the push_back method of the AVL tree.
call_bulk(
    [
        checker.mark_for_liquidation(
            (self.client.key.public_key_hash(), burrow_no)
        )
        for burrow_no in burrows
    ],
    batch_size=40,
)

# This touch starts a liquidation auction
#
# This would use the split method of the AVL tree. However, if the entire queue
# has less tez than the limit, the 'split' method would trivially return the entire
# tree without much effort; so here we should ensure that the liquidation queue has
# more tez than the limit (FIXME).
call_endpoint(checker, "touch", None)

# And we place a bid

ret = checker.metadata.currentLiquidationAuctionMinimumBid().storage_view()
auction_id, minimum_bid = ret["contents"], ret["nat_1"]
# The return value is supposed to be annotated as "auction_id" and "minimum_bid", I
# do not know why we get these names. I think there is an underlying pytezos bug
# that we should reproduce and create a bug upstream.

# TODO: Send another slice to auction and cancel it. Since we have filled up the auction queue above 
# and the current auction is descending, we should be able to reliably cancel the slice.

# Note the auction ptr for later operations
current_auctions_ptr = checker.storage["deployment_state"]["sealed"]["liquidation_auctions"]["current_auction"]()["contents"]

# Place a bid on the current auction
call_endpoint(
    checker, "liquidation_auction_place_bid", (auction_id, minimum_bid)
)
level_after_bid = self.client.shell.head.level()

Sending 1 operations as bulk: Batch 1 of 1


MichelsonError: ({'id': 'proto.009-PsFLoren.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 42,
  'with': {'int': '90'}},)

In [5]:
level_after_bid = self.client.shell.head.level()

In [ ]:
# New stuff to add to tests

# Restore index
#
# Keep in mind that we're using a patched checker on tests where the protected index
# is much faster to update.
call_endpoint(oracle, "update", 1_000_000)

In [19]:
# Once max_bid_interval_in_blocks blocks have passed, the liquidation auction we 
# bid on should be complete.
slept = 0
while self.client.shell.head.level() < level_after_bid + 20:
    sleep_for = 2
    time.sleep(sleep_for)
    slept += sleep_for
    if slept > 40:
        raise Exception(
            "Chain is growing at a much slower rate than this test expects. "
            "Check the sandbox to ensure that it is healthy and running with the expected configuration.")

# Touch Checker to update auction state. This should move the current auction which we bid on
# to the completed auctions list.
call_endpoint(checker, "touch", None)

# Touch liquidation slices to update slice state and free up our winnings
# TODO: touch slices

# Claim our winnings
# TODO: liquidation_auction_claim_win

Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch with None


  {'kind': 'big_map',
         'id': '15',
         'diff': {'action': 'update', 'updates': []}},
        {'kind': 'big_map',
         'id': '14',
         'diff': {'action': 'update', 'updates': []}},
        {'kind': 'big_map',
         'id': '13',
         'diff': {'action': 'update', 'updates': []}},
        {'kind': 'big_map',
         'id': '12',
         'diff': {'action': 'update', 'updates': []}},
        {'kind': 'big_map',
         'id': '11',
         'diff': {'action': 'update', 'updates': []}},
        {'kind': 'big_map',
         'id': '10',
         'diff': {'action': 'update', 'updates': []}}]}},
     {'kind': 'transaction',
      'source': 'KT1GosH9zvtJwPFwGpwMbznmxxPwUZtu2B5B',
      'nonce': 3,
      'amount': '0',
      'destination': 'KT1XdEm8gFeh58M2uKbNLtCbJKPXmkoreYZ3',
      'parameters': {'entrypoint': 'burrowSendSliceToChecker',
       'value': {'int': '97901000'}},
      'result': {'status': 'applied',
       'storage': {'prim': 'Pair',
        'args': [{'p

2026

In [88]:
# TODO:
# get auction id of current auction when placing bid
# add logic for reducing total auction time in e2e src patch
# Touch slices and reclaim bid

# TODO: Get this programatically above
current_auctions_ptr = 2013

In [95]:
def avl_storage(ptr):
    return checker.storage["deployment_state"]["sealed"]["liquidation_auctions"]["avl_storage"]["mem"][ptr]()

In [102]:
auction_slices = []






avl_storage(avl_storage(current_auctions_ptr)["root"][0])

{'branch': {'left': 101,
  'left_height': 6,
  'left_tez': 2839129000,
  'parent': 2013,
  'right': 165,
  'right_height': 7,
  'right_tez': 3734336000}}

In [122]:
def auction_avl_leaves(avl_ptr):
    node = avl_storage(avl_ptr)
    node_type = list(node.keys())[0]
    
    if node_type == "root":
        next_node, _ = node["root"]
        # Tree is empty
        if next_node is None:
            return []
        else:
            for ptr, leaf_node in auction_avl_leaves(next_node):
                yield ptr, leaf_node
    elif node_type == "leaf":
        yield avl_ptr, node
    else:
        # We are in a branch
        # Traverse left side first
        for ptr, leaf_node in auction_avl_leaves(node["branch"]["left"]):
            yield ptr, leaf_node        
        # Then the right side
        for ptr, leaf_node in auction_avl_leaves(node["branch"]["right"]):
            yield ptr, leaf_node

In [124]:
auctioned_slices = set([])
for leaf_ptr, leaf in auction_leaves(current_auctions_ptr):
    # Double check that we are only working with leaf ptrs
    assert "leaf" in leaf
    auctioned_slices.add(leaf_ptr)

In [127]:
# Note: gas exhausted when calling with all slices
# FIXME: What is the current max length of list we can pass here without exceeding
# the gas limit?
to_touch = list(auctioned_slices)

In [131]:
touched = []
for i in range(len(to_touch)):
    slice_ptr = to_touch[i]
    if i in touched:
        continue
    call_endpoint(checker, "touch_liquidation_slices", [slice_ptr])

Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [132]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [136]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [140]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [144]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [148]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [152]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [156]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [160]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [164]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [168]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [172]
Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [176]
Call

In [139]:
print(avl_storage(current_auctions_ptr)["root"][0])

128


In [140]:
call_endpoint(checker, "touch_liquidation_slices", [128])
avl_storage(current_auctions_ptr)["root"]

Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / touch_liquidation_slices with [128]


(None,
 {'older_auction': None,
  'sold_tez': 10000000000,
  'winning_bid': {'address': 'tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6',
   'kit': 30066325422},
  'younger_auction': None})

In [145]:
call_endpoint(checker, "liquidation_auction_claim_win", current_auctions_ptr)

Calling tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6 / liquidation_auction_claim_win with 2013


{'protocol': 'PsFLorenaUUuikDWvMDr6fGBRG8kt3e3D3fHoXK1j1BFRxeSH4i',
 'chain_id': 'NetXfpUfwJdBox9',
 'hash': 'opFGqPYoJhuE254R8jbZZYZEPqMzuyEt5ZNjtaJdEbjmrwizGP3',
 'branch': 'BL59kL1r5y7RFnhZqx4c7vLAWYSa5QJvjswed7pzCw9PMb6RuPT',
 'contents': [{'kind': 'transaction',
   'source': 'tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6',
   'fee': '5325',
   'counter': '6202',
   'gas_limit': '50236',
   'storage_limit': '100',
   'amount': '0',
   'destination': 'KT1EbVVPit28YGES6pcihRJWJrLtxQrycbbS',
   'parameters': {'entrypoint': 'liquidation_auction_claim_win',
    'value': {'int': '2013'}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6',
      'change': '-5325',
      'origin': 'block'},
     {'kind': 'freezer',
      'category': 'fees',
      'delegate': 'tz1YPSCGWXwBdTncK2aCctSZAXWvGsGwVJqU',
      'cycle': 785,
      'change': '5325',
      'origin': 'block'}],
    'operation_result': {'status': 'applied',
     'storage': {'prim'